In [1]:
import tensorflow as tf
import numpy as np

In [3]:
# 在内存中生成模拟数据
def GenerateData(datasize=100):
    train_X = np.linspace(-1,1,datasize)
    train_Y = 2 * train_X + np.random.randn(*train_X.shape) * 0.3 
    return train_X,train_Y

train_data = GenerateData() 

In [5]:
train_data

(array([-1.        , -0.97979798, -0.95959596, -0.93939394, -0.91919192,
        -0.8989899 , -0.87878788, -0.85858586, -0.83838384, -0.81818182,
        -0.7979798 , -0.77777778, -0.75757576, -0.73737374, -0.71717172,
        -0.6969697 , -0.67676768, -0.65656566, -0.63636364, -0.61616162,
        -0.5959596 , -0.57575758, -0.55555556, -0.53535354, -0.51515152,
        -0.49494949, -0.47474747, -0.45454545, -0.43434343, -0.41414141,
        -0.39393939, -0.37373737, -0.35353535, -0.33333333, -0.31313131,
        -0.29292929, -0.27272727, -0.25252525, -0.23232323, -0.21212121,
        -0.19191919, -0.17171717, -0.15151515, -0.13131313, -0.11111111,
        -0.09090909, -0.07070707, -0.05050505, -0.03030303, -0.01010101,
         0.01010101,  0.03030303,  0.05050505,  0.07070707,  0.09090909,
         0.11111111,  0.13131313,  0.15151515,  0.17171717,  0.19191919,
         0.21212121,  0.23232323,  0.25252525,  0.27272727,  0.29292929,
         0.31313131,  0.33333333,  0.35353535,  0.3

In [6]:
# 将内存中数据转化为数据集
dataset = tf.data.Dataset.from_tensor_slices(train_data)  # 以元组的方式生成数据集

dataset2 = tf.data.Dataset.from_tensor_slices({
    "x":train_data[0],
    "y":train_data[1] 
})

In [7]:
batchsize = 10    #定义批次样本个数
dataset3 = dataset.repeat().batch(batchsize)   # 按批次组合数据集

dataset4 = dataset2.map(lambda data:
            (data['x'],tf.cast(data['y'],tf.int32)))  # 转化数据集中的元素
dataset5 = dataset.shuffle(100)     # 乱序数据集

In [9]:
def getone(dataset):
    iterator = dataset.make_one_shot_iterator()    # 生成一个迭代器
    one_element = iterator.get_next()
    return one_element

one_element1 = getone(dataset)
one_element2 = getone(dataset2)
one_element3 = getone(dataset3)
one_element4 = getone(dataset4)
one_element5 = getone(dataset5) 

In [10]:
one_element1

(<tf.Tensor 'IteratorGetNext:0' shape=() dtype=float64>,
 <tf.Tensor 'IteratorGetNext:1' shape=() dtype=float64>)

In [11]:
one_element2

{'x': <tf.Tensor 'IteratorGetNext_1:0' shape=() dtype=float64>,
 'y': <tf.Tensor 'IteratorGetNext_1:1' shape=() dtype=float64>}

In [15]:
def showone(one_element,datasetname):
    print('{0:-^50}'.format(datasetname))
    for ii in range(5):
        datav = sess.run(one_element)
        print(datasetname,"-",ii,"| x,y:",datav)

In [16]:
def showbatch(onebatch_element,datasetname):
    print("{0:-^50}".format(datasetname))
    for ii in range(5):
        datav = sess.run(onebatch_element)
        print(datasetname,"-",ii,"| x.shape:",np.shape(datav[0]),"| x[:3]",datav[0][:3])
        print(datasetname,"-",ii,"| y.shape:",np.shape(datav[1]),"| y[:3]",datav[1][:3])

In [17]:
with tf.Session() as sess:
    showone(one_element1,"dataset1")
    showone(one_element2,"dataset2")
    showbatch(one_element3,"dataset3")
    showone(one_element4,"dataset4")
    showone(one_element5,"dataset5")

---------------------dataset1---------------------
dataset1 - 0 | x,y: (-1.0, -2.0035687821035917)
dataset1 - 1 | x,y: (-0.9797979797979798, -1.7318519968991652)
dataset1 - 2 | x,y: (-0.9595959595959596, -1.4396291568860693)
dataset1 - 3 | x,y: (-0.9393939393939394, -2.3418485397853614)
dataset1 - 4 | x,y: (-0.9191919191919192, -2.0330586775078214)
---------------------dataset2---------------------
dataset2 - 0 | x,y: {'x': -1.0, 'y': -2.0035687821035917}
dataset2 - 1 | x,y: {'x': -0.9797979797979798, 'y': -1.7318519968991652}
dataset2 - 2 | x,y: {'x': -0.9595959595959596, 'y': -1.4396291568860693}
dataset2 - 3 | x,y: {'x': -0.9393939393939394, 'y': -2.3418485397853614}
dataset2 - 4 | x,y: {'x': -0.9191919191919192, 'y': -2.0330586775078214}
---------------------dataset3---------------------
dataset3 - 0 | x.shape: (10,) | x[:3] [-1.         -0.97979798 -0.95959596]
dataset3 - 0 | y.shape: (10,) | y[:3] [-2.00356878 -1.731852   -1.43962916]
dataset3 - 1 | x.shape: (10,) | x[:3] [-0.797

> 值得注意的是：在使用tf.data.Dataset.from_tensor_slices接口时，如果传入的是列表类型对象，则系统将其中的元素当作数据处理；如如果传入的是元组类型对象，则将其中的对象当作列来拆开。这是一个值得注意的地方。

In [18]:
# 传入的对象是列表
dataset1 = tf.data.Dataset.from_tensor_slices([1,2,3,4,5]) 
def getone(dataset):
    iterator = dataset.make_one_shot_iterator()
    one_element = iterator.get_next()
    return one_element

one_element1 = getone(dataset1)

with tf.Session() as sess:
    for i in range(5):
        print(sess.run(one_element1)) 

1
2
3
4
5


In [19]:
# 传入的对象是元组
dataset2 = tf.data.Dataset.from_tensor_slices((1,2,3,4,5))
def getone(dataset):
    iterator = dataset.make_one_shot_iterator()
    one_element = iterator.get_next()
    return one_element 

one_element2 = getone(dataset2)

with tf.Session() as sess:
    for i in range(5):
        print(sess.run(one_element2))  

IndexError: list index out of range

In [20]:
# 解决办法是：将数据中每个数字改成数组，即可解决上述问题
dataset3 = tf.data.Dataset.from_tensor_slices(([1],[2],[3],[4],[5]))

def getone(dataset):
    iterator = dataset.make_one_shot_iterator()
    one_element = iterator.get_next()
    return one_element

one_element3 = getone(dataset3)

with tf.Session() as sess:
    for i in range(5):
        print(sess.run(one_element3))  

(1, 2, 3, 4, 5)


OutOfRangeError: End of sequence
	 [[node IteratorGetNext_6 (defined at <ipython-input-20-845ce981e427>:6)  = IteratorGetNext[output_shapes=[[], [], [], [], []], output_types=[DT_INT32, DT_INT32, DT_INT32, DT_INT32, DT_INT32], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_6)]]

Caused by op 'IteratorGetNext_6', defined at:
  File "/Users/lidianxiang/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/lidianxiang/anaconda3/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/lidianxiang/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/lidianxiang/anaconda3/envs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/lidianxiang/anaconda3/envs/tensorflow/lib/python3.6/s